In [1]:
import random
import math
from time import sleep, time
from itertools import count
from collections import deque, namedtuple
from tqdm.notebook import trange, tqdm
try:
    import gym
except ModuleNotFoundError:
    !pip install gym
    !pip install ale-py
    !pip install gym[atari]
    !pip install gym[accept-rom-license]
    import gym
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch import nn
from torchvision import transforms
dev = 'cuda' if torch.cuda.is_available() else 'cpu'
to_tensor = lambda x: torch.tensor(x, device=dev)

In [2]:
env = gym.make("ALE/Pong-v5")
state = env.reset()

A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]
/usr/local/lib/python3.9/dist-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
# Play
def play(policy, runs=50, wait=0):
    for _ in trange(runs):
        state = env.reset()
        for _ in count():
            state, reward, done, info = env.step(policy(state))
            plt.imshow(env.render(mode='rgb_array'))

            if done:
                break
            sleep(wait)

In [4]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward', 'done'))

class ReplayMemory:
    def __init__(self, capacity):
        self.queue = deque([], maxlen=capacity)
        
    def __len__(self):
        return len(self.queue)
        
    def push(self, *args):
        self.queue.append(Transition(*args))

    def sample(self, batchsize):
        return random.sample(self.queue, batchsize)

In [5]:
def save(net, losses, rewards):
    torch.save({
        'net': net.state_dict(),
        'losses': losses,
        'rewards': rewards
    }, 'chkpnt.pt')

def load():
    chkpnt = torch.load('./chkpnt.pt')
    return chkpnt['net'], chkpnt['losses'], chkpnt['rewards']

In [6]:
class DQN(nn.Module): # size((210, 160, 3)) -> size(6)
    def __init__(self):
        super(DQN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 8, 4),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, 4, 2),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.l = nn.Sequential(
            nn.Linear(3136, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, env.action_space.n)
        )
        self.flat = nn.Flatten()
        self.transformations = transforms.Compose([
            transforms.ToTensor(),
            transforms.Grayscale(),
            transforms.Resize((110, 84)),
            transforms.CenterCrop((84, 84))
        ])
    
    def transforms(self, x):
        return self.transformations(x).to(dev)
    
    def forward(self, x):
        if len(x) < 4:
            x = x.unsqueeze(0)
        x = self.conv(x)
        x = self.flat(x)
        return self.l(x)


net = DQN().to(dev)
target_net = DQN().to(dev)
memory = ReplayMemory(10000)
losses = []
rewards = []
try:
    state_dict, losses, rewards = load()
    net.load_state_dict(state_dict)
except FileNotFoundError:
    pass
target_net.load_state_dict(net.state_dict())
# net(net.transforms(state)).shape

<All keys matched successfully>

In [7]:
def optimize_model(memory, net, target_net, criterion, optim):
    # Update weights
    transition_batch = memory.sample(BS)

    states = to_tensor(np.array([(batch[0]) for batch in transition_batch]))
    next_states = torch.cat(
        [net.transforms(batch[2]).unsqueeze(0) for batch in transition_batch])
    future_q = target_net(next_states)
    
    Y = net(states)
    for i, (state, action, next_state, reward, done) in enumerate(transition_batch):
        reward = to_tensor(reward)
    
        y = reward if done else reward + gamma*torch.max(future_q[i])
        Y[i][action] = y

    for p in net.parameters():
        p.grad = None
    loss = criterion(Y, net(states))

    loss.backward()
    optim.step()
    return loss.item()

In [ ]:
n_episodes = 1000
gamma = 0.9
BS = 64
C = 5 # update target

EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 50
get_eps = lambda steps_done: EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)

criterion = nn.MSELoss()
optim = torch.optim.RMSprop(net.parameters(), lr=1e-4)

steps_done = 0
target_count = 0
for episode in (tr := trange(len(rewards), n_episodes)):
    rewards.append(0)
    state = net.transforms(env.reset())
    for t in count():
        eps_threshold = get_eps(steps_done)
        steps_done += 1
        
        if np.random.uniform(0, 1) < eps_threshold:
            action = env.action_space.sample()
        else:
            action = torch.argmax(net(state)).item()
        next_state, reward, done, info = env.step(action)
        rewards[episode] += reward
        memory.push(state.cpu().numpy(), action, next_state, reward, done)

        if len(memory) >= BS:
            loss = optimize_model(memory, net, target_net, criterion, optim)
            losses.append(loss)
            tr.set_description(f'Loss {loss:.4f}, Reward {rewards[-1]:04.1f}, Step {t:04d}')
        else:
            tr.set_description(f'Reward {rewards[-1]:04.1f}, Step {t:04d}')

        state = net.transforms(next_state)
        if target_count >= C:
            target_net.load_state_dict(net.state_dict())
            target_count = 0
        else:
            target_count += 1
        if done:
            break
    save(net, losses, rewards)

  0%|          | 0/997 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


In [ ]:
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(15, 6, forward=True)

axs[0].plot(losses, label='losses')
axs[1].plot(rewards, label='rewards')

[ax.legend() for ax in axs]
np.array(losses[-100:]).mean()

In [ ]:
def policy(state):
     return torch.argmax(net(net.transforms(state))).item()
play(policy, runs=10, wait=0)